# Project 3 Behavioural Cloning
## Background


## Rationale

## Plan

## Conclusion



### Load Data

In [1]:
import cv2
import csv
import numpy as np
from PIL import Image

In [2]:
def process_image(image):
    return image

In [3]:
def prepImage(path):
    source_path = path
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    return cv2.imread(current_path, 1) # 0 = grayscale, 1 = Colour


In [4]:

lines = []
with open('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

lines = lines[1:]
        
images = []
measurements = []
for line in lines:
    # IMAGES
    img_center = prepImage(line[0])
    images.append(img_center)
    
    img_left = prepImage(line[1])
    images.append(img_left)
    
    img_right = prepImage(line[2])
    images.append(img_right)
    
    # STEERING
    steering_center = float(line[3])
    measurements.append(steering_center)
    
    correction = 0.2 # this is a parameter to tune
    steering_left = steering_center + correction
    measurements.append(steering_left)
    steering_right = steering_center - correction
    measurements.append(steering_right)
    


In [5]:
# csv_file = './data/driving_log.csv'
# car_images = []
# steering_angles = []
# with open(csv_file, 'r') as f:
#     reader = csv.reader(f)
#     next(reader, None)  # skip the headers
#     for row in reader:
#         steering_center = float(row[3])

#         # create adjusted steering measurements for the side camera images
#         correction = 0.2 # this is a parameter to tune
#         steering_left = steering_center + correction
#         steering_right = steering_center - correction

#         # read in images from center, left and right cameras
#         path = './data/' # fill in the path to your training IMG directory
#         img_center = process_image(np.asarray(Image.open(path + row[0].replace(' ',''))))
#         img_left = process_image(np.asarray(Image.open(path + row[1].replace(' ',''))))
#         img_right = process_image(np.asarray(Image.open(path + row[2].replace(' ',''))))

#         # add images and angles to data set
#         car_images.extend(img_center)
#         car_images.extend(img_left)
#         car_images.extend(img_right)
#         steering_angles.extend(steering_center)
#         steering_angles.extend(steering_left)
#         steering_angles.extend(steering_right)

        

In [6]:
        
# Flip the Images for more data   
augmented_images, augmented_measurements = [] ,[]
for image, measurement in zip(images, measurements ):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)  
        
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)   

del(images)
del(measurements)
del(augmented_images)
del(augmented_measurements)

In [11]:
import sys
sys.getsizeof(X_train) / 1e9

NameError: name 'images' is not defined

In [ ]:
# Backup
# lines = []
# with open('./data/driving_log.csv') as csvfile:
#     reader = csv.reader(csvfile)
#     for line in reader:
#         lines.append(line)

# lines = lines[1:]
        
# images = []
# measurements = []
# for line in lines:
#     source_path = line[0]
#     filename = source_path.split('/')[-1]
#     current_path = './data/IMG/' + filename
#     image = cv2.imread(current_path, 1) # 0 = grayscale, 1 = Colour
    
#     # Preprocess with CLAHE (Contrast Limited Adaptive Histogram Equalization)
#     # http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html
# #     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# #     image = clahe.apply(image)
    
#     images.append(image)
#     measurement = float(line[3])
#     measurements.append(measurement)
    
# X_train = np.array(images)
# y_train = np.array(measurements)    


In [13]:
# ## Split into train and validation

# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
import gc
gc.collect()

166

In [8]:
from __future__ import print_function
import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [9]:
# input image dimensions
img_rows, img_cols = 160, 320
# Colour
input_shape = (img_rows, img_cols, 3)
# Grayscale
#input_shape = (img_rows, img_cols, 1)

In [ ]:
# FIX  RANK <4 issue
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
    
img = X_train[0]
imgplot = plt.imshow(img)

X_train = X_train.astype('float32').reshape((5625, 160, 320, 3))
X_valid = X_valid.astype('float32').reshape((2411, 160, 320, 3))
# X_train /= 255
# X_valid /= 255
print('x_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'validation samples')

In [ ]:
# # Save Processed Data
# import pickle
# with open('X_train.p','wb') as myfile:
#     pickle.dump(X_train, myfile)   

# with open('X_valid.p','wb') as myfile:
#     pickle.dump(X_valid, myfile)   
    
# with open('y_train.p','wb') as myfile:
#     pickle.dump(y_train, myfile)  

# with open('y_valid.p','wb') as myfile:
#     pickle.dump(y_valid, myfile)   


### Image Augmentation

In [10]:
from keras.preprocessing.image import ImageDataGenerator

# Create Data Generator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True) #,
    #rotation_range=20, # Could cause an issue for steering
    #width_shift_range=0.2, 
    #height_shift_range=0.2 ) #,
    # horizontal_flip=True) # need to flip steering too

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)




MemoryError: 

In [ ]:
# Recommended Flipping
from keras.layers import Lambda
from keras.layers import Cropping2D





In [15]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5),
    ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, verbose=0)
]

batch_size = 256
epochs = 5
learn_rate = 0.001
drop_rate = 0.3

model = Sequential()
# Normalise
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# Crop
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

# Conv Layer 1
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(drop_rate))

# Conv Layer 2
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(drop_rate))

# Conv Layer 3
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(drop_rate))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(1))

model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=learn_rate))

# fits the model on batches with real-time data augmentation:
model.fit_generator(generator=datagen.flow(X_train, y_train, batch_size=batch_size), 
                    validation_data=datagen.flow(X_valid, y_valid, batch_size=batch_size),
                    #validation_split=0.2,
                    shuffle=True,
                    steps_per_epoch=len(X_train) / batch_size, 
                    validation_steps=len(X_valid) / batch_size,
                    epochs=epochs,
                    callbacks=callbacks)

# Save model in callbacks
# model.save('model.h5')

NameError: name 'EarlyStopping' is not defined

In [ ]:
## BACKUP
# callbacks = [
#     EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=5),
#     ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, verbose=0)
# ]

# batch_size = 256
# epochs = 5
# learn_rate = 0.001
# drop_rate = 0.3

# model = Sequential()
# # Normalise
# model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# # Crop
# model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(3,160,320)))

# # Conv Layer 1
# model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(drop_rate))

# # Conv Layer 2
# model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(drop_rate))

# # Conv Layer 3
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(drop_rate))

# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(drop_rate))
# model.add(Dense(1))

# model.compile(loss='mse', optimizer=keras.optimizers.Adam(lr=learn_rate))

# # Model without Image Augmentation
# # model.fit(X_train, y_train,
# #           batch_size=batch_size,
# #           epochs=epochs,
# #           verbose=1,
# #           validation_data=(X_valid, y_valid),
# #           shuffle=True,
# #           callbacks=callbacks)
# # model.save('model.h5')

# # fits the model on batches with real-time data augmentation:
# model.fit_generator(generator=datagen.flow(X_train, y_train, batch_size=batch_size), 
#                     validation_data=datagen.flow(X_valid, y_valid, batch_size=batch_size),
#                     steps_per_epoch=len(X_train) / batch_size, 
#                     validation_steps=len(X_valid) / batch_size,
#                     epochs=epochs,
#                     callbacks=callbacks)

# # Save model in callbacks
# # model.save('model.h5')